# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: descobrir o produto mais barato e atualizar isso em uma planilha.
- Caso o valor seja 20% (ou mais) abaixo do preço original, queremos também ser avisados por e-mail para poder agir rápido e aproveitar essa promoção.

- No nosso caso, vamos fazer com produtos comuns em sites como Amazon, Magazine Luiza e Lojas Americanas, mas a ideia é a mesma.

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos e o link em cada loja, além do preço original cadastrado.

### O que devemos fazer:

- Cadastrar na coluna Preço Atual o menor preço encontrado e na Coluna Local o nome do Local onde foi encontrado esse preço
- Enviar um e-mail para compras com a notificação do menor preço encontrado e o link de compra, caso o preço encontrado esteja com 20% ou mais de desconto em relação ao preço original. (Vou usar o e-mail pythonimpressionador+compras@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

### Adicional:

- Podemos colocar esse programa para rodar de 4 em 4 horas ou então todo dia as 10hrs da manhã. Podemos fazer isso via agendador de tarefas do Windows ou então deixar o código rodando em background com um time.sleep(tempo)
- Caso queira, você pode deixar o navegador sem aparecer ao término do seu código, para ficar mais sutil o seu programa

In [197]:
import pandas as pd
from selenium import webdriver

In [198]:
desconto_min = 0.1
produtos_df = pd.read_excel(r'Produtos.xlsx')
produtos_df = produtos_df.fillna('-')
display(produtos_df)
print(produtos_df['Amazon'][0])

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,4277.07,Lojas Americanas
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/LG-50UN731C-Assisten...,https://www.americanas.com.br/produto/3120097572,https://www.magazineluiza.com.br/smart-tv-lg-l...,2900,2415.92,Amazon


https://www.amazon.com.br/IPHONE-11-PRETO-TELA-C%C3%82MERA/dp/B086VQTMPR/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=iphone+11&qid=1606856566&sr=8-1


In [199]:
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
driver = webdriver.Chrome()
driver.set_window_position(-10000, 0)


def tratar_texto(texto):
     return float(texto.replace('R$','').replace('.','').replace(',','.'))


In [200]:
# Passos
# Entrar no site da Amazon
# Descobrir o objeto aonde está o preço
# Armazenar o preço em uma variável

In [201]:
for i, linha in produtos_df.iterrows():
    
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_class_name('priceBlockSavingsString').text
    except:
        preco_amazon = driver.find_element_by_css_selector('#olp_feature_div span.a-size-base').text
    preco_amazon = tratar_texto(preco_amazon)
    
    # Entrar no site da Lojas Americanas
    driver.get(linha['Lojas Americanas'])
    try:
        preco_lame = driver.find_element_by_class_name('src__BestPrice-sc-1jvw02c-5').text
        preco_lame = tratar_texto(preco_lame)
    except:
        print('Produto {} não disponível nas Lojas Americanas'.format(linha['Link Produto']))
        preco_lame = linha['Preço Original']*3
    
        # Entrar no site do Magazine Luiza
    driver.get(linha['Magazine Luiza'])
    try:
        preco_magalu = driver.find_element_by_class_name('price-template__text').text
        preco_magalu = tratar_texto(preco_magalu)
    except:
        print('Produto {} não disponível na loja Magazine Luiza'.format(linha['Link Produto']))
        preco_magalu = linha['Preço Original']*3
    preco_original = linha['Preço Original']
    lista_precos = [(preco_amazon, 'Amazon'), (preco_lame, 'Lojas Americanas'), (preco_magalu, 'Magazine Luiza'), (preco_original, 'Preço Original')]
    lista_precos.sort()
    print(linha['Link Produto'])
    print(lista_precos[0])
    
    produtos_df.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produtos_df.loc[i, 'Local'] = lista_precos[0][1]
    
    if lista_precos[0][0] <= preco_original*(1 - desconto_min):
        enviar_email = True
driver.quit()

iPhone 11 Apple 64GB Preto
(4277.07, 'Lojas Americanas')
Produto Smart TV LED 50'' LG Ultra HD 4K Thinq AI não disponível nas Lojas Americanas
Smart TV LED 50'' LG Ultra HD 4K Thinq AI
(2415.92, 'Amazon')


In [202]:
display(produtos_df)

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,4277.07,Lojas Americanas
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/LG-50UN731C-Assisten...,https://www.americanas.com.br/produto/3120097572,https://www.magazineluiza.com.br/smart-tv-lg-l...,2900,2415.92,Amazon


In [203]:
# salvar arquivo

produtos_df.to_excel('Produtos.xlsx', index=False)



In [204]:
#enviar email
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

# filtrar a tabela de produtos
tabela_filtrada = produtos_df.loc[produtos_df['Preço Atual'] <= produtos_df['Preço Original']*(1 - desconto_min), :]
if enviar_email:
    mail = outlook.CreateItem(0)
    mail.To = 'mfsoaresig+1@gmail.com'
    mail.CC = 'marso2001@uol.com.br'
    mail.BCC = 'soares.marcos@icloud.com'
    mail.Subject = 'URGENTE - Menor preço detectado'
    mail.HTMLBody = f'<p>Esse são os produtos com mais de {desconto_min:.0%} de desconto</p>{tabela_filtrada.to_html()}'
    
    mail.Send()
print('Email enviado')

Email enviado
